In [77]:

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

from sympy.solvers import solve
from sympy import Symbol
import math
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from numpy import linalg as LA

import time

In [78]:
def function(x,E,theta,eta,w,delta,alpha,s_0,b,rho):

    lambda_ = alpha*(x[4]+s_0)**b
    
    
    A = x[2]*x[3]/w
    mu = 25
    
    #dot_W = x[0]*rho*((mu/x[2]-1)+float(np.random.normal(0,0.22,1)))
    dot_W = x[0]*rho*((mu/x[2]-1))
    #dot_W = 0
    dot_L = theta*(lambda_*(A-x[1])-A)
    
    #dot_L = theta*(lambda_*(E*lambda_-x[1])-E*lambda_ )
    
    dot_p = (x[2]* (1-x[3]) * dot_W / x[0] + eta*(w-x[0])*(E-A+x[1])+ w*dot_L ) / (1-x[3]*w-(1-x[3])*x[0])
    #dot_p = ( eta*(w-x[0])*(E-A+x[1])+ w*dot_L ) 
    
    
    # / (1-x[3]*w-(1-x[3])*x[0])
    dot_n = x[3]* (-(1-w)*dot_p/x[2] + 1/A*(eta*(E-A+x[1])+dot_L))
    #dot_n = 0
    
    #dot_s = -delta*(x[4]) + delta * (np.log(1-dot_p/x[2]))**2
    #dot_s = -delta*(x[4]) + delta * (np.log(1-dot_p/x[2]))**2
    dot_s = -delta*(x[4]) + delta * (-0.1*dot_p/x[2])**2
    
    
    return([dot_W, dot_L, dot_p, dot_n, dot_s])
   

In [79]:
def sys(iv, dt, time,function,E,theta,eta,w,delta,alpha,s_0,b,rho):
    n = int(time/dt)
    t = np.linspace(0,time,n)
    x = np.array([[ i for i in iv]]*n)
    n_ord = len(iv)
    
    a = iv
    
    for i in range(1,n):
        X = function(a,E,theta,eta,w,delta,alpha,s_0,b,rho)
        X[0] = X[0] 
        a = a + np.array(X)*dt
        x[i] = a
    return np.array(x)


In [83]:
rho= 0.1
#rho = 100
E = 2
mu = 25
theta = 10
eta = 10
w = 0.5

delta = 2
alpha = 0.01
#alpha = 0.1

s_0 = 10**-6
b = -0.5

In [86]:
T = 300
dt = 10**-2
delta = 0.5

start_time = time.time()

p0 = 20
L0 = 20
x= sys([0.3,L0,p0,0.5,1], dt,T,function,E,theta,eta,w,delta,alpha,s_0,b,rho)

print("--- %s seconds ---" % (time.time() - start_time))


n = int(T/dt)
t = np.linspace(0,T,n)

--- 0.35243797302246094 seconds ---


In [89]:
fig = plt.figure(figsize=(5,5))
fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax1 = fig.add_subplot(5,1,1)
ax2 = fig.add_subplot(5,1,2)
ax3 = fig.add_subplot(5,1,3)
ax4 = fig.add_subplot(5,1,4)
ax5 = fig.add_subplot(5,1,5)

n = int(T/dt)
t = np.linspace(0,T,n)

#lambda_ = alpha * (x[:,4] + s_0)**b
w = 0.5
#A = x[:,2]*x[:,3]/w

T1 = 0
T2 = 20000
#xx= np.array([x[500*i] for i in range(int(len(x)//1000))])
#t = np.array([t[500*i] for i in range(int(len(t)//1000))])
xx = x
lambda_ = alpha * (xx[:,4] + s_0)**b

tau = 1
#ax1.plot(t,10*(A-x[:,1])/x[:,2], 'r-',label ='W')
ax1.plot(tau*t,xx[:,0], 'k-',label ='W')
ax1.set_ylabel(r'$W$')


ax2.plot(tau*t,xx[:,1], 'k-',label ='L')
#ax2.plot(t,x[:,1]-(1-1/lambda_)*x[:,2]*x[:,3]/w, 'c-',label ='L')
#ax2.plot(t,x[:,1]-(lambda_-1)*E, 'b-',label ='L')
ax2.set_ylabel(r'$L$')


ax3.plot(tau*t,xx[:,2], 'k-',label ='p')
# num = (1-n)*rho*mu+eta*(w-W)*(E+L)-w*theta*lambda_*L
# denom = (1-n)*rho + eta*(w-W)*n/w -theta*n*(lambda_-1)
# ax3.plot(t,num/denom,'k--',alpha=0.5)
ax3.set_ylabel(r'$p$')


ax4.plot(tau*t,xx[:,3], 'k-',label ='n')
#ax4.plot(t,w/x[:,2] * (E+x[:,1]), 'r--',alpha=0.4)
ax4.set_ylabel(r'$n$')


#ax5.plot(t,A/E, 'k--',label ='s',alpha=0.5)
ax5.plot(tau*t,lambda_, 'k-',label ='s')

#ax5.plot(t,(np.log(1-theta*lambda_*(E*(lambda_-1)-x[:,1]) / (x[:,1]+10)))**2, 'g--',label ='s')
ax5.set_ylabel(r'$ \lambda $')
ax5.set_xlabel(r'$ t $')


plt.grid()
plt.show()


## Eigenvalues

In [25]:
def eigenvalue_dis(value):
    tau = dt
    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.075,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,s = value
    
    Mat = np.zeros((5,5))
    A = n*p / w
    lambda_ = alpha*(s+s0)**bb
    
    Mat[0,0] = rho*(mu/p-1)
    Mat[0,2] = -W*rho*mu/p**2

    Mat[1,1] = -theta*lambda_
    Mat[1,2] = theta*(lambda_-1)*n/w
    Mat[1,3] = theta*(lambda_-1)*p/w
    Mat[1,4] = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) + (1-n)*dot_p / (1-n*w-(1-n)*W)**2
    df3dl = ( (w-W)*eta + w*Mat[1,1] ) / (1-w*n-(1-n)*W)
    df3dp =  ( (1-n)*(-rho)-(w-W)*eta*n/w + w*Mat[1,2] ) / (1-w*n-(1-n)*W)
    df3dn =  ( -rho*(mu-p)-(w-W)*eta*p/w + w*Mat[1,3] ) / (1-w*n-(1-n)*W) - (-w+W)*((1-n)*rho*(mu/p-1)*eta*(E-A+L)+w*dot_L) / (1-w*n-(1-n)*W)**2
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)

    Mat[2,0] = df3dW
    Mat[2,1] = df3dl
    Mat[2,2] = df3dp
    Mat[2,3] = df3dn
    Mat[2,4] = df3dlambda

    
    df4dl = w/p*(eta+Mat[1,1])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+Mat[1,2])
    df4dn = w/p*(-eta*p/w+Mat[1,3])
    df4dlambda = w/(n*p)*theta*(A-L)

    Mat[3,0] = -(1-w)*n/p * df3dW
    Mat[3,1] = -(1-w)*n/p * df3dl + df4dl
    Mat[3,2] = -(1-w)*n * (df3dp/p-dot_p/p**2) + df4dp
    Mat[3,3] = -(1-w)/p *(dot_p + n*df3dn) + df4dn
    Mat[3,4] = -(1-w)*n/p * df3dlambda + df4dlambda


    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))



    Mat[4,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dW /p
    Mat[4,1] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dl /p
    Mat[4,2] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dp /p - dot_p/p**2)
    Mat[4,3] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    Mat = tau * Mat
    for i in range(5):
        Mat[i,i] = 1 + Mat[i,i]
    eigva, v = LA.eig(Mat)
    return(eigva,v)

In [26]:
def eigenvalue(value):

    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.075,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,s = value
    
    Mat = np.zeros((5,5))
    A = n*p / w
    lambda_ = alpha*(s+s0)**bb
    
    Mat[0,0] = rho*(mu/p-1)
    Mat[0,2] = -W*rho*mu/p**2

    Mat[1,1] = -theta*lambda_
    Mat[1,2] = theta*(lambda_-1)*n/w
    Mat[1,3] = theta*(lambda_-1)*p/w
    Mat[1,4] = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)+eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    #df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) - (1-n)*dot_p / (1-n*w-(1-n)*W)**2
    df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) + (1-n)*dot_p / (1-n*w-(1-n)*W)**2 # change sign
    df3dl = ( (w-W)*eta + w*Mat[1,1] ) / (1-w*n-(1-n)*W)
    df3dp =  ( (1-n)*(-rho)-(w-W)*eta*n/w + w*Mat[1,2] ) / (1-w*n-(1-n)*W)
    df3dn =  ( -rho*(mu-p)-(w-W)*eta*p/w + w*Mat[1,3] ) / (1-w*n-(1-n)*W) - (-w+W)*((1-n)*rho*(mu-p)+eta*(E-A+L)+w*dot_L) / (1-w*n-(1-n)*W)**2
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)

    Mat[2,0] = df3dW
    Mat[2,1] = df3dl
    Mat[2,2] = df3dp
    Mat[2,3] = df3dn
    Mat[2,4] = df3dlambda

    
    df4dl = w/p*(eta+Mat[1,1])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+Mat[1,2])
    df4dn = w/p*(-eta*p/w+Mat[1,3])
    df4dlambda = w/p*theta*(A-L)

    Mat[3,0] = -(1-w)*n/p * df3dW
    Mat[3,1] = -(1-w)*n/p * df3dl + df4dl
    Mat[3,2] = -(1-w)*n * (df3dp/p-dot_p/p**2) + df4dp
    Mat[3,3] = -(1-w)/p *(dot_p + n*df3dn) + df4dn
    Mat[3,4] = -(1-w)*n/p * df3dlambda + df4dlambda


    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))



    #Mat[4,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dW /p
    #Mat[4,1] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dl /p
    #Mat[4,2] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dp /p - dot_p/p**2)
    #Mat[4,3] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    #Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    Mat[4,0] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dW /p
    Mat[4,1] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dl /p
    Mat[4,2] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * (df3dp /p - dot_p/p**2)
    Mat[4,3] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * (df3dn /p)
    Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 - tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    eigva, v = LA.eig(Mat)
    return(eigva,v)

In [27]:
list_v = np.copy(x)

In [28]:
def resort(x):
    list_v = np.array([eigenvalue(value)[0] for value in x])
    for i in range(len(x)):
        order = [0,0,0,0,0]
        eig = eigenvalue(x[i])
        order[0],order[1] = eig[0].imag.argsort()[0],eig[0].imag.argsort()[-1]
        order[2],order[3] = eig[0].real.argsort()[0],eig[0].real.argsort()[1]
        order[4] = list(set(([0,1,2,3,4])) - set((order[0:4])))[0]
        order = np.array(order)
        list_v[i] = eig[0][order]
    return(list_v)

## Summer modif 

In [29]:
#list_values = resort(x)
list_values = np.array([eigenvalue(value)[0] for value in x])

In [30]:
T1 = 0
T2 = 20000

In [31]:
plt.figure(figsize=(41,41))
for i in range(5):
    plt.plot(t[T1:T2],list_values[:,i][T1:T2],'.',markersize=0.1)
plt.grid()

/home/lcbob/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/home/lcbob/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/home/lcbob/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/home/lcbob/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/home/lcbob/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, ord

In [32]:
eig_vects = eigenvalue(x[5550])[1]

for i in range(len(eig_vects)):
    print(eig_vects[:,i])

[-2.05248015e-04-2.98229966e-05j -6.65908140e-01+1.31461889e-02j
 -7.45777429e-01+0.00000000e+00j -1.41502362e-02+3.04519225e-03j
 -9.09234058e-05+8.82161632e-07j]
[-2.05248015e-04+2.98229966e-05j -6.65908140e-01-1.31461889e-02j
 -7.45777429e-01-0.00000000e+00j -1.41502362e-02-3.04519225e-03j
 -9.09234058e-05-8.82161632e-07j]
[ 6.08501952e-03+0.j -2.85237863e-02+0.j  9.99573875e-01+0.j
 -1.19796215e-03+0.j  3.78905623e-06+0.j]
[-0.01019267+0.j -0.0229152 +0.j  0.99968317+0.j -0.00107591+0.j
  0.0018467 +0.j]
[ 0.01042444+0.j  0.48810341+0.j -0.83378287+0.j  0.01178668+0.j
  0.25751425+0.j]


In [33]:
eigenvalue(x[5550])[0]

array([-10.02473853+1.45860207j, -10.02473853-1.45860207j,
        -0.44974333+0.j        ,   0.29034474+0.j        ,
         0.23929766+0.j        ])

In [36]:
T1 = 0
T2 = 20000
lambda_ = alpha*(x[:,4]+s_0)**b
print(len(x),T1,T2,len(t),len(lambda_))


30000 0 20000 30000 30000


In [70]:
fig = plt.figure(figsize=(8,8))


for i in range(5):
    print(i)
    fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
    ax1 = fig.add_subplot(3,1,1)
    ax2 = fig.add_subplot(3,1,2)
    ax3 = fig.add_subplot(3,1,3)
    
    ax1.plot(t[T1:T2],lambda_[T1:T2], 'g-',label ='s',markersize=0.1)
        
    ax2.plot(t[T1:T2],list_values[:,i][T1:T2].real,'+--',markersize=1,alpha=0.5)
    #ax2.plot(tt[T1:T2],0+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],-1+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],np.absolute(list_values[:,i][T1:T2]),'-',alpha=0.5)
    
    ax3.plot(t[T1:T2],list_values[:,i][T1:T2].imag,'.',markersize=0.5)

ax1.set_ylabel('$\lambda$')        
ax2.set_ylabel('$Re(\lambda_i)$')        
ax3.set_ylabel('$Imag(\lambda_i)$')         
ax3.set_xlabel('$t$')     

plt.show()

0
1
2
3
4


/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  import sys
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
/home/lcbob/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reus

In [18]:
len(x)

30000

In [15]:
w,v = eigenvalue(x[int(14000)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-11.105960400037466+11.98555558844652j) 
 [-7.88765710e-05-8.50625395e-05j -6.96186134e-01+1.14389987e-01j
 -7.07545208e-01+0.00000000e+00j -1.26562519e-02+3.81003335e-02j
 -2.78320003e-03-2.04560521e-05j]
(-11.105960400037466-11.98555558844652j) 
 [-7.88765710e-05+8.50625395e-05j -6.96186134e-01-1.14389987e-01j
 -7.07545208e-01-0.00000000e+00j -1.26562519e-02-3.81003335e-02j
 -2.78320003e-03+2.04560521e-05j]
(1.6747941916300277+0j) 
 [ 0.00118529+0.j  0.67539118+0.j -0.73721697+0.j -0.01870966+0.j
 -0.00253645+0.j]
(-0.011201969154594836+0j) 
 [ 0.05528734+0.j -0.91655178+0.j  0.39561769+0.j  0.01888229+0.j
  0.00250141+0.j]
(0.4485088514210197+0j) 
 [ 6.04282043e-03+0.j  1.14977672e-01+0.j -9.93346509e-01+0.j
 -2.48600754e-03+0.j -3.90299383e-04+0.j]


In [40]:
w,v = eigenvalue(x[int(2860)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-365797.8308187609+0j) 
 [-3.03705356e-13+0.j -5.46725487e-05+0.j -5.94831281e-05+0.j
 -8.89165793e-07+0.j  9.99999997e-01+0.j]
(0.18823106890795385+1.2288172037474299j) 
 [ 1.70279220e-04-0.0008996j   7.09458616e-01+0.j
 -6.09730072e-01-0.01776397j  1.68034093e-02+0.00011816j
 -3.52196508e-01+0.01601767j]
(0.18823106890795385-1.2288172037474299j) 
 [ 1.70279220e-04+0.0008996j   7.09458616e-01-0.j
 -6.09730072e-01+0.01776397j  1.68034093e-02-0.00011816j
 -3.52196508e-01-0.01601767j]
(-0.633128866519828+51.766056549309205j) 
 [ 1.45380067e-05-5.04209151e-06j -1.29057281e-01-3.68999726e-01j
 -1.34881304e-01-4.04638127e-01j  9.57512765e-04-5.98756071e-03j
 -8.15611226e-01+0.00000000e+00j]
(-0.633128866519828-51.766056549309205j) 
 [ 1.45380067e-05+5.04209151e-06j -1.29057281e-01+3.68999726e-01j
 -1.34881304e-01+4.04638127e-01j  9.57512765e-04+5.98756071e-03j
 -8.15611226e-01-0.00000000e+00j]


In [46]:
w,v = eigenvalue(x[int(2862)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-15703.40558640296+0j) 
 [-1.64809928e-10+0.j -1.27166021e-03+0.j -1.38339432e-03+0.j
 -2.07208290e-05+0.j  9.99998234e-01+0.j]
(0.00414572149142315+0.8619668911141425j) 
 [-2.15034778e-05+0.00160566j -6.37786425e-01+0.03422485j
  7.39928883e-01+0.j         -1.67223266e-02+0.00069028j
  2.08764702e-01-0.02640061j]
(0.00414572149142315-0.8619668911141425j) 
 [-2.15034778e-05-0.00160566j -6.37786425e-01-0.03422485j
  7.39928883e-01-0.j         -1.67223266e-02-0.00069028j
  2.08764702e-01+0.02640061j]
(-4.347478290365381+22.075522015427264j) 
 [-3.88094280e-05+3.01205483e-05j  2.47583912e-01+4.72995629e-01j
  2.65387325e-01+5.27825343e-01j -2.31604009e-04+8.02346619e-03j
  6.04882539e-01+0.00000000e+00j]
(-4.347478290365381-22.075522015427264j) 
 [-3.88094280e-05-3.01205483e-05j  2.47583912e-01-4.72995629e-01j
  2.65387325e-01-5.27825343e-01j -2.31604009e-04-8.02346619e-03j
  6.04882539e-01-0.00000000e+00j]


In [90]:

w,v = eigenvalue(x[int(2863)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-8160121.898026429+0j) 
 [ 5.76133101e-16+0.j  2.45737795e-06+0.j  2.93134362e-06+0.j
  3.13539077e-08+0.j -1.00000000e+00+0.j]
(1.2275039622186275+1.5817907326982443j) 
 [-3.07171537e-04+0.00035563j -4.88462881e-01-0.00929019j
  5.89329425e-01+0.02673378j -1.21423200e-02-0.0003417j
  6.42764395e-01+0.j        ]
(1.2275039622186275-1.5817907326982443j) 
 [-3.07171537e-04-0.00035563j -4.88462881e-01+0.00929019j
  5.89329425e-01-0.02673378j -1.21423200e-02+0.0003417j
  6.42764395e-01-0.j        ]
(22.65405295038444+102.18292653999696j) 
 [-4.39459315e-06-3.85749950e-07j  3.67062433e-02+2.40685722e-01j
  3.75470078e-02+2.85444266e-01j -9.78189479e-04+2.58800868e-03j
  9.26189599e-01+0.00000000e+00j]
(22.65405295038444-102.18292653999696j) 
 [-4.39459315e-06+3.85749950e-07j  3.67062433e-02-2.40685722e-01j
  3.75470078e-02-2.85444266e-01j -9.78189479e-04-2.58800868e-03j
  9.26189599e-01-0.00000000e+00j]


In [68]:

w,v = eigenvalue(x[int(4200)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-699.2147524594908+0j) 
 [-1.12104809e-07+0.j -2.89399528e-02+0.j -3.47981175e-02+0.j
 -5.56225842e-04+0.j  9.98975105e-01+0.j]
(-0.13199421352982577+0.8932381296302329j) 
 [ 3.29720847e-04+0.00226495j -3.80344940e-01+0.02120602j
  9.17183875e-01+0.j         -1.16103310e-02+0.00063337j
  1.13254833e-01-0.02635667j]
(-0.13199421352982577-0.8932381296302329j) 
 [ 3.29720847e-04-0.00226495j -3.80344940e-01-0.02120602j
  9.17183875e-01-0.j         -1.16103310e-02-0.00063337j
  1.13254833e-01+0.02635667j]
(-7.576483797765469+9.90033511863931j) 
 [-8.13267570e-05-1.06298730e-04j -5.66138484e-01+5.02996457e-03j
 -7.40667820e-01+0.00000000e+00j -8.28244681e-03-5.65508901e-03j
 -2.66029412e-01+2.44950715e-01j]
(-7.576483797765469-9.90033511863931j) 
 [-8.13267570e-05+1.06298730e-04j -5.66138484e-01-5.02996457e-03j
 -7.40667820e-01-0.00000000e+00j -8.28244681e-03+5.65508901e-03j
 -2.66029412e-01-2.44950715e-01j]


In [247]:
print(669/0.121 ,-255.9/-0.245 )

5528.925619834711 1044.4897959183675


### Test _Slow-Fast

In [199]:
l_a0 = np.linspace(-4,4,300)
l_a1 = np.linspace(-1,1,50)
#l_a1 = np.logspace(np.log(0.1)/np.log(10), np.log(120)/np.log(10), 100)
# a1 = 1
# root_real = [np.real([np.roots([-a1**2,-2*a0*a1,-a0**2-s_0,0,alpha**2 ])[i] for a0 in l_a0]) for i in range(4)]
# root_imag = np.imag([np.roots([-a1**2,-2*a0*a1,-a0**2-s_0,0,alpha**2 ]) for a0 in l_a0])


In [200]:
def pos_or_nan(x):
    if x >= 0:
        return(x)
    else:
        return(np.nan)
    
def lim_or_nan(x,c):
    if abs(x) < c:
        return(x)
    else:
        return(np.nan)

In [230]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

a0,a1 = l_a0[0],l_a1[0]

#color = ['b','r','g','k','y']
color = ['b']*5

opacity = [0.2]*5
for a1 in l_a1:
    root_real = [np.real([np.roots([-(a1)**2,2*a0*(a1),-a0**2-s_0,0,alpha**2,0 ])[i] for a0 in l_a0]) for i in range(len(np.roots([-(-a1+a0)**2,2*a0*(-a1+a0),-a0**2-s_0,0,alpha**2,0 ])))]
    for i in range(len(root_real)):
        z = [pos_or_nan(j) for j in root_real[i]]
        ax.plot(l_a0,[a1]*len(l_a0),z,c=color[i],alpha=opacity[i])
        
ax.set_zlim3d(0.5,40)

plt.show()

In [235]:
T1,T2 = int(50/dt),int(150/dt)

W,L,p,n,s = np.transpose(x)
A = n*p /w
# x_a0 = rho*(mu/p-1)
# x_a1 = -rho*(mu/p-1)*E*p/w

x_a0 = - (rho*(mu/p-1)*(1-n) + 1/p *eta*(w-W)*(E-n*p/w+L)- theta *n) 
x_a1 = (theta*w*(A-L)) / (p)

T1, T2 = 5,60
ax.plot(x_a0[T1:T2],x_a1[T1:T2],lambda_[T1:T2],'g-',alpha=0.8)
ax.plot([x_a0[T1]],[x_a1[T1]],[lambda_[T1]],'r.',alpha=0.5)

T1,T2 = int(50/dt),int(150/dt)
ax.plot(x_a0[T1:T2],x_a1[T1:T2],lambda_[T1:T2],'k-',alpha=0.5)


x_a0 =  theta *n 
x_a1 = (theta*n*(E/(E+L)))
#ax.plot(x_a0[T1:T2],x_a1[T1:T2],lambda_[T1:T2],'k.-',alpha=0.5)        
        
        
# x_a0 = (plt.plot(delta*x[:,4])
# ax.plot(x_a0[T1:T2],x_a1[T1:T2],lambda_[T1:T2],'k-',alpha=0.8)
# ax.set_zlim3d(-1,20)

ax.set_xlabel('$a_0$')
ax.set_ylabel('$a_1$')
ax.set_zlabel('$\lambda$')

plt.show()


In [210]:

x_a0 =  theta *n
#x_a1 = theta*w*(A-L)/p
x_a1 = theta*(-W/p*(E-A+L)+w*E/p+(1-n)*rho*(mu/p-1))

# x_a0 = (rho*(mu/p-1)*(1-n) + 1/p *eta*(w-W)*(E-n*p/w+L)- theta *n) 
# x_a1 = (theta*w*(p*n/w-L)) / (p)

ax.plot(x_a0[T1:T2],x_a1[T1:T2],lambda_[T1:T2],'k-',alpha=0.5)
plt.show()

In [65]:
plt.figure()
# plt.plot(eta*(w-W)*(E-n*p/w+L))
# plt.plot(L-(1-1/lambda_)*A)
# plt.plot(x[:,4])
# plt.plot(lambda_)
#plt.plot(delta*x[:,4])
#plt.plot( ((rho*(mu/p-1)*(1-n)+1/p * eta*(w-W)*(E-A+L) + theta*w/p*(lambda_*(A-L)-A)) / (1-n*w-(1-n)*W))**2)

dotp = ((rho*(mu/p-1)*(1-n)+1/p * eta*(w-W)*(E-A+L) + theta*w/p*(lambda_*(A-L)-A)) / (1-n*w-(1-n)*W))
plt.plot( ((rho*(mu/p-1)*(1-n)*p+eta*(w-W)*(E-A+L) + theta*w*(lambda_*(A-L)-A)) / (1-n*w-(1-n)*W)))
plt.plot(theta*(lambda_*(A-L)-A))
plt.plot((-(1-w)*dotp/p+1/A*(eta*(E-A+L)+theta*(lambda_*(A-L)-A))))
plt.plot((-delta*(-s+(dotp/p)**2)))
plt.plot(n*dotp+eta*(E-A+L)+theta*(lambda_*(A-L)-A),'k-')
plt.plot(rho*(mu/p-1),'y')
plt.show()

## Stability on the slow Manifold

In [91]:
def derivative(z,x_a0,x_a1):
    lambda_ = np.array(z)
    alpha =0.1
    s_0 = 10**-6
    der = (delta)*(-5*lambda_**4*x_a1**2 -4*lambda_**3*2*x_a0*x_a1 -3* lambda_**2*(x_a0**2+s_0) + alpha**2)
    return(der)
    

In [94]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

a0,a1 = l_a0[0],l_a1[0]
for a1 in l_a1:
    root_real = [np.real([np.roots([-a1**2,-2*a0*a1,-a0**2-s_0,0,alpha**2 ])[i] for a0 in l_a0]) for i in range(len(np.roots([-a1,-2*a0*a1,-a0-s_0,0,alpha ])))]
    for i in range(len(root_real)):
        z = [pos_or_nan(j) for j in root_real[i]]
        y = np.array([a1]*len(l_a0))
        ax.plot(l_a0,y,derivative(z,l_a0,y),'m-',alpha=0.5)
plt.show()

In [95]:

lim_der = 5000

der = (delta)*(-5*lambda_**4*x_a1**2 -4*lambda_**3*2*x_a0*x_a1 -3* lambda_**2*(x_a0**2+s_0) + alpha**2)[T1:T2]
der = [lim_or_nan(x,lim_der) for x in der]
ax.plot(x_a0[T1:T2],x_a1[T1:T2],der,'k-.',alpha=0.8,markersize=0.5,linewidth=0.4)

ax.set_zlim3d(-20,20)

plt.show()

25000